In [2]:
import numpy as np
from sklearn import neighbors

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
datapath = '/media/sf_VBox_Shared/timeseries/UCR_WaveGesture/'

In [6]:
X_train = np.load(datapath+'X_train.npy')
y_train_binary = np.load(datapath+'y_train_binary.npy')
X_val = np.load(datapath+'X_val.npy')
y_val_binary = np.load(datapath+'y_val_binary.npy')
X_test = np.load(datapath+'X_test.npy')
y_test_binary = np.load(datapath+'y_test_binary.npy')

In [12]:
def transform_dataset(X):
    num_samples, num_timesteps, num_channels = X.shape
    return X.reshape(num_samples, num_timesteps*num_channels)

In [8]:
n_neighbors = 1

In [10]:
clf = neighbors.KNeighborsClassifier(n_neighbors)

In [13]:
clf.fit(transform_dataset(X_train), y_train_binary)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [14]:
val_predict = clf.predict(transform_dataset(X_val))

In [15]:
val_predict

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [18]:
from sklearn import metrics
metrics.accuracy_score(val_predict, y_val_binary)

0.93308550185873607

In [19]:
metrics.log_loss(val_predict, y_val_binary)

2.3111448888787884